<a href="https://colab.research.google.com/github/songqsh/foo1/blob/master/src/dp_hjb_cauchy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

__ref__ 

- notes on dp - [pdf](https://github.com/songqsh/foo1/blob/master/doc/190531note.pdf)

In [0]:
# paras

DIM = 2 #Dimension for state space;

TERM = 1 #Terminal time

#We want the value function on the hybercube domain [LOW, HIGH]^DIM
LOW = -1

HIGH = 1

NUM = 2 #discrete parameter, usually number of meshes in one space unit



#Define action space on the hybercube domain [LOW, HIGH]^DIM
#assume action space has the same dim as state

LOW_A = -1

HIGH_A = 1

NUM_A = 4 #discrete parameter, usually number of meshes in one unit in action space


In [0]:
#paras

lambda_ = 1 # lambda is python keyword, so use lambda_ instead

#fun_g is terminal cost
#input x is an array with length = DIM
def fun_g(x):
  return np.log((np.sum(x**2)+1)/2)

#fun_l is running cost
#input t (1-d), x (DIM-d), a (DIM-d)
def fun_l(t, x, a):
  return np.sum(a**2)
  
#fun_b is drift
#input t (1-d), x (DIM-d), a (DIM-d)

def fun_b(t,x,a):
  return 2*np.sqrt(lambda_)*a

In [0]:
# paras

h = 1/NUM #mesh size in state space

delta = h**2/2 #mesh size in time space

n_time = int(TERM/delta)+1 # number of times

n_one_state = int((HIGH-LOW)/h+1) # number of states in one axis

h_a = 1/NUM_A #mesh size in action space

n_one_action = int((HIGH_A - LOW_A)/h_a +1) # number of actions along one axis

n_step = 2*DIM+1 

In [0]:
#initialize q_value and state value as zero
state_shape = n_one_state*np.ones(DIM, dtype = int)
action_shape = n_one_action*np.ones(DIM, dtype = int)

time_state_shape = np.insert(state_shape, 0, n_time) 
time_state_action_shape = np.append(time_state_shape, action_shape) 

q_val = np.zeros(tuple(time_state_action_shape), dtype = float)
s_val = np.zeros(tuple(time_state_shape), dtype = float)


In [0]:
def s_q_sync(s_val, q_val, i):
  s_val_buf = s_val[i]
  q_val_buf = q_val[i]
  it = np.nditer(s_val_buf, flags = ['multi_index'], op_flags = ['readwrite'])
  with it:
    while not it.finished:
      ind_ = it.multi_index 
      it[0] = np.min(q_val_buf[ind_])
      it.iternext()
      
  s_val[i] = s_val_buf

In [0]:
# set q_value at the terminal time as the terminal cost

q_val_buf = q_val[-1]

it = np.nditer(q_val_buf, flags=['multi_index'], op_flags=['readwrite'])
with it:
  while not it.finished:
    ind_ = it.multi_index[0:DIM] #array index for current state
    x = LOW + h*np.array(ind_) #actual coordinate for the current state
    it[0] = fun_g(x) #update q_val_buf
    it.iternext()
    
q_val[-1] = q_val_buf #update q_val
s_q_sync(s_val, q_val, -1) #sync s_val and q_val

In [0]:
'''
This function returns next time_state_ind 
by taking current time_state_ind and 
step_ind.

Inputs:
  time_state: DIM+1 array
  time_state[0]: time ranging from 0 to n_time-1
  time_state[i] for i >=1 : state on i-axis ranging from 0 to n_one_sate-1
  step_ind: step index ranging from 0 to n_step -1
output:
  next_state: DIM+1 array
'''

def post_state(time_state, step_ind):
  next_state = np.array(time_state)
  next_state[0] += 1
  if step_ind > 0:
    n_axis = int((step_ind+1)/2)
    step_size = 1
    if np.mod(step_ind,2) == 0: 
      step_size = -1
    next_state[n_axis] += step_size
    
    #if move out from above, go flat
    next_state[n_axis] = np.min([np.max([next_state[n_axis], 0]), n_one_state-1])
      
  return next_state

In [0]:
'''
Inputs:
  time_state: DIM+1 array
  time_state[0]: time ranging from 0 to n_time-1
  time_state[i] for i >=1 : state on i-axis ranging from 0 to n_one_sate-1
output:
  actual_state: DIM+1 array
'''

def coordinate(time_state):
  return np.array(time_state[0])*delta, LOW + np.array(time_state[1:])*h

In [0]:
'''
Inputs:
  time: scalar
  state: DIM-d array
  action: DIM-d array
output:
  running cost: a scalar
  policy: a prob on step_ind space of length 2*DIM+1  
'''

def policy_fun(time, state, action):  
  b_ = fun_b(time, state, action)
  
  policy = np.zeros(n_step)
  
  
  
  for i in range(1,n_step):
    axis_n = int((i+1)/2)
    
    if np.mod(i,2) == 1:
      policy[i] = (np.max([0, b_[axis_n-1]])*h+1)*delta/(h**2)
    else:
      policy[i] = (np.max([0, -b_[axis_n-1]])*h+1)*delta/(h**2)
  
  policy[0] = np.max([1 - np.sum(policy), 0]) 
  
  if policy[0] == 0:
    policy = policy/np.sum(policy)

  return policy

In [0]:
# update q_value and s_value backward in time

for t_ind in range(n_time-2,-1,-1):
  time = t_ind*delta  
  q_val_buf = q_val[t_ind]
  

  it = np.nditer(q_val_buf, flags=['multi_index'], op_flags=['readwrite'])
  with it:
    while not it.finished:
      ind_ = it.multi_index[0:DIM] #array index for current state
      state = LOW + h*np.array(ind_) #actual coordinate for the current state
      a_ind_ = it.multi_index[DIM:] #array index for current action
      action = LOW_A + np.array(a_ind_)*h_a #actual action value
      #update q_val_buf      
      tmp1 = fun_l(time, state, action) #loss
      policy = policy_fun(time, state, action)
      for i in range(len(policy)):
        now_ind_ = np.append([t_ind], ind_)
        next_ind_ = post_state(now_ind_, i)
        
        tmp1 += s_val[tuple(next_ind_)]*policy[i]
      it[0] = tmp1  
      it.iternext()
    
  q_val[t_ind] = q_val_buf#update q_val
  s_q_sync(s_val, q_val, t_ind) #sync s_val and q_val

In [0]:
print(s_val[0])

[[-0.02973043 -0.0411894  -0.04840844 -0.0411894  -0.02973043]
 [-0.0411894  -0.0528793  -0.05974552 -0.0528793  -0.0411894 ]
 [-0.04840844 -0.05974552 -0.0671953  -0.05974552 -0.04840844]
 [-0.0411894  -0.0528793  -0.05974552 -0.0528793  -0.0411894 ]
 [-0.02973043 -0.0411894  -0.04840844 -0.0411894  -0.02973043]]
